# Invisibility Cloak

## Defining Functions

In [1]:
import cv2
import numpy as np

def create_mask(frame, lower_hsv, upper_hsv, kernel_size=3):
	inspect = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
	mask = cv2.inRange(inspect, lower_hsv, upper_hsv)
	mask = cv2.medianBlur(mask, 3)
	kernel = np.ones((kernel_size, kernel_size), np.uint8)
	mask = cv2.dilate(mask, kernel, iterations=5)

	return mask

def create_trackbars():
	cv2.createTrackbar("lower_hue", "Bars", 0, 180, nothing)
	cv2.createTrackbar("lower_saturation", "Bars", 0, 255, nothing)
	cv2.createTrackbar("lower_value", "Bars", 0, 255, nothing)
	cv2.createTrackbar("upper_hue", "Bars", 0, 180, nothing)
	cv2.createTrackbar("upper_saturation", "Bars", 0, 255, nothing)
	cv2.createTrackbar("upper_value", "Bars", 0, 255, nothing)

def get_trackbar_values():
	lower_value = cv2.getTrackbarPos("lower_value", "Bars")
	lower_hue = cv2.getTrackbarPos("lower_hue", "Bars")
	lower_saturation = cv2.getTrackbarPos("lower_saturation", "Bars")
	upper_hue = cv2.getTrackbarPos("upper_hue", "Bars")
	upper_saturation = cv2.getTrackbarPos("upper_saturation", "Bars")
	upper_value = cv2.getTrackbarPos("upper_value", "Bars")

	return lower_hue, lower_saturation, lower_value, upper_hue, upper_saturation, upper_value

def nothing(x):
	pass

## Launch the cam

Press `q` to quit.

Slider values to detect red:

Lower -> 150H, 90S, 0V

Upper -> 180H, 255S, 255V (max)

In [6]:
# Set up the cam
cap = cv2.VideoCapture(0)
bars = cv2.namedWindow("Bars")

create_trackbars()

image_size = (120, 240, 3)
l_center = (image_size[1] // 4, image_size[0] // 2)
radius = min(l_center[0], l_center[1]) - 5

# Capture the initial frame for creating the background
while True:
	cv2.waitKey(1000)
	ret, init_frame = cap.read()
	if ret:
		break

prev_lh, prev_ls, prev_lv, prev_uh, prev_us, prev_uv = [0, 0, 0, 179, 255, 255]

# Start the cloak cam
while True:
	ret, frame = cap.read()

	lower_hue, lower_saturation, lower_value, upper_hue, upper_saturation, upper_value = get_trackbar_values()
	lower_hsv = np.array([lower_hue, lower_saturation, lower_value])
	upper_hsv = np.array([upper_hue, upper_saturation, upper_value])

	mask = create_mask(frame, lower_hsv, upper_hsv)

	mask_inv = 255 - mask

	# Bitwise operations to get the final frame
	frame_inv = cv2.bitwise_and(frame, frame, mask=mask_inv)

	# Bitwise operations to get the blanket area
	blanket_area = cv2.bitwise_and(init_frame, init_frame, mask=mask)

	# Combine the two frames
	final = cv2.bitwise_or(frame_inv, blanket_area)

	if (prev_lh, prev_ls, prev_lv, prev_uh, prev_us, prev_uv) != (lower_hue, lower_saturation, lower_value, upper_hue, upper_saturation, upper_value):
		circs = np.ones(image_size, dtype=np.uint8) * 255

		for y in range(image_size[0]):
			for x in range(image_size[1] // 2):
				r_sq = (x - l_center[1]) ** 2 + (y - l_center[0]) ** 2
				if r_sq < radius ** 2:
					hue = round((np.arctan2(y - l_center[0], x - l_center[1]) / np.pi + 1) * 90)
					sat = round(np.sqrt(r_sq) / radius * 255)
					if lower_hue <= hue <= upper_hue and lower_saturation <= sat <= upper_saturation:
						circs[y, x] = cv2.cvtColor(np.uint8([[(hue, sat, lower_value)]]), cv2.COLOR_HSV2BGR)[0][0]
						circs[y, x + image_size[1] // 2] = cv2.cvtColor(np.uint8([[(hue, sat, upper_value)]]), cv2.COLOR_HSV2BGR)[0][0]

	cv2.imshow('Detection Spectra', circs)

	cv2.imshow("Cloak Cam", final)

	if cv2.waitKey(3) == ord('q'):
		break

	prev_lh, prev_ls, prev_lv, prev_uh, prev_us, prev_uv = (lower_hue, lower_saturation, lower_value, upper_hue, upper_saturation, upper_value)

cv2.destroyAllWindows()
cap.release()